<a href="https://colab.research.google.com/github/kallebcintra/Artificial-Intelligence/blob/main/Atividade%20A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pdb;

# Exercício 1

Qual é o caminho mais rápido para ir de 1 a N, sabendo que as únicas duas ações possíveis são:
 - andar de i para i + 1, com duração de 1 minuto (para todo i de 1 a N - 1)
 - mágica de i para 2*i, com duração de 2 minutos (para todo i de 1 a N/2)

Implemente as funções da classe abaixo que define a interface desse problema para utilização com algoritmos de busca vistos em sala de aula. Em sequência, escolha e implemente um dos algoritmos de busca explicados em sala que resolva o problema. Faça essa implementação na função *solution*, que tem como input um objeto criado utilizando a classe implementada *TransportationProblem*.

In [ ]:
class TransportationProblem:
    def __init__(self, N):
        self.N = N
    def startState(self):
        return 1
    def isEnd(self, state):
        return state == self.N
    def succAndCost(self, state):
        # retorna uma lista de tuples com 3 informações:
        # (ação, novo estado, custo)
        # TODO
        if state*2 <= self.N:
            costs_list = [('walk', state+1, 1), ('magic', state*2, 2)]
            return costs_list

        costs_list = [('walk', state+1, 1)]
        return costs_list

In [ ]:
def solution(problem):
    # path = []
    # TODO
    # return path


    min_costs = {}
    paths = {}

    def dp(state):
        if state in min_costs:
            return min_costs[state]

        if problem.isEnd(state):
            min_costs[state] = 0
            paths[state] = []
            return 0

        actions = problem.succAndCost(state)
        min_cost = float('inf')
        min_path = []

        for action in actions:
            _, next_state, cost = action
            future_cost = dp(next_state)
            total_cost = cost + future_cost

            if total_cost < min_cost:
                min_cost = total_cost
                min_path = [action] + paths[next_state]

        min_costs[state] = min_cost
        paths[state] = min_path
        return min_cost

    start_state = problem.startState()
    shortest_cost = dp(start_state)
    shortest_path = paths[start_state]

    return shortest_path

Exemplos:


> N = 2 \
> p = TransportationProblem(N) \
> print(*solution(p), sep='\n') \

('walk', 2, 1)

=====================================

> N = 3 \
> p = TransportationProblem(N) \
> print(*solution(p), sep='\n') \

('walk', 2, 1) \
('walk', 3, 1)

=====================================

> N = 10 \
> p = TransportationProblem(N) \
> print(*solution(p), sep='\n') \

('walk', 2, 1) \
('magic', 4, 2) \
('walk', 5, 1) \
('magic', 10, 2) \

ou

('walk', 2, 1) \
('walk', 3, 1) \
('walk', 4, 1) \
('walk', 5, 1) \
('magic', 10, 2) \

(**nesse caso, os caminhos apresentam o mesmo custo total, logo não é necessário retornar as duas soluções, basta uma delas**)

=====================================

> N = 12 \
> p = TransportationProblem(N) \
> print(*solution(p), sep='\n') \

('walk', 2, 1) \
('walk', 3, 1) \
('magic', 6, 2) \
('magic', 12, 2) \

In [ ]:
N = 100 # teste a sua solução para valores de N diferentes dos exemplos e veja se os resultados fazem sentido
p = TransportationProblem(N)
path = solution(p)
# print(*solution(p), sep='\n')
print(*path, sep='\n')

('walk', 2, 1)
('walk', 3, 1)
('magic', 6, 2)
('magic', 12, 2)
('magic', 24, 2)
('walk', 25, 1)
('magic', 50, 2)
('magic', 100, 2)


Implemente uma soução para o problema com o algoritmo A* e uma solução com Dynamic Programing. Compare em gráfico o tempo de execução dos dois algoritmos e da sua solução (caso ela tenha sido diferente) para N variando de 10 a 500 (caso esteja demorando muito para Ns maiores, vá até o maior valor de N que conseguir).

In [ ]:
# def solutionAstar(problem):
    # path = []
    # TODO
    # return path

# def solutionDP(problem):
    # path = []
    # TODO
    # return path


from queue import PriorityQueue

def solutionAstar(problem):
    start_state = problem.startState()
    start_cost = 0
    queue = PriorityQueue()
    queue.put((start_cost, [], start_state))

    cost_table = {start_state: start_cost}

    if isinstance(problem, RestrictedTransportationProblem):
        def heuristic(state):
            num, odd_count = state
            if num * 2 <= problem.N:
                return problem.N - (num * 2) + 2
            else:
                return problem.N - num
    else:
        def heuristic(state):
            if state * 2 <= problem.N:
                return problem.N - (state * 2) + 2
            else:
                return problem.N - state

    while not queue.empty():
        current_cost, current_path, current_state = queue.get()

        if problem.isEnd(current_state):
            return current_path

        successors = problem.succAndCost(current_state)

        for action, next_state, cost in successors:
            new_cost = current_cost + cost

            if next_state not in cost_table or new_cost < cost_table[next_state]:
                cost_table[next_state] = new_cost

                heuristic_value = heuristic(next_state)

                priority = new_cost + heuristic_value

                new_path = current_path + [(action, next_state, cost)]
                queue.put((priority, new_path, next_state))

    return None

In [ ]:
N = 23
p = TransportationProblem(N)
print(*solutionAstar(p), sep='\n')

('walk', 2, 1)
('magic', 4, 2)
('walk', 5, 1)
('magic', 10, 2)
('walk', 11, 1)
('magic', 22, 2)
('walk', 23, 1)


In [ ]:
def solutionDP(problem):
    min_costs = {}
    paths = {}

    def dp(state):
        if state in min_costs:
            return min_costs[state]

        if problem.isEnd(state):
            min_costs[state] = 0
            paths[state] = []
            return 0

        actions = problem.succAndCost(state)
        min_cost = float('inf')
        min_path = []

        for action in actions:
            _, next_state, cost = action
            future_cost = dp(next_state)
            total_cost = cost + future_cost

            if total_cost < min_cost:
                min_cost = total_cost
                min_path = [action] + paths[next_state]

        min_costs[state] = min_cost
        paths[state] = min_path
        return min_cost

    start_state = problem.startState()
    shortest_cost = dp(start_state)
    shortest_path = paths[start_state]

    return shortest_path

In [ ]:
N = 23
p = TransportationProblem(N)
print(*solutionDP(p), sep='\n')

('walk', 2, 1)
('walk', 3, 1)
('walk', 4, 1)
('walk', 5, 1)
('magic', 10, 2)
('walk', 11, 1)
('magic', 22, 2)
('walk', 23, 1)


**Dica para medir o tempo de execução de uma função:**

Mais informações no [link da documentação](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit).

In [ ]:
def dummyFunc(N = 10):
    s = sum(range(N))

t = %timeit -q -o dummyFunc()
#t = %timeit -n 1 -q -o dummyFunc() # para o caso do comando acima estar demorando muito

t.average

N = 100 # teste a sua solução para valores de N diferentes didx = calculated_cost(costs_list[0][1])os exemplos e veja se os resultados fazem sentido
p = TransportationProblem(N)

t1 = %timeit -q -o solutionDP(p)
print("t1: ", t1.average)

t2 = %timeit -q -o solutionAstar(p)
print("t2: ", t2.average)

t1:  0.0001596031066000023
t2:  0.00017733082467142851


# Exercício 2:

Qual é o caminho mais rápido para ir de 1 a N passando obrigatoriamente por pelo menos 3 números ímpares, novamente sabendo que as duas únicas ações possíveis são:
 - andar de i para i + 1, com duração de 1 minuto (para todo i de 1 a N - 1)
 - mágica de i para 2*i, com duração de 2 minutos (para todo i de 1 a N/2)

Implemente uma nova classe *RestrictedTransportationProblem* para este problema e aplique os algoritmos propostos nas funções *solution*, *solutionAstar* e *solutionDP* para resolvê-lo.

In [ ]:
class RestrictedTransportationProblem:
    def __init__(self, N):
        self.N = N

    def startState(self):
        # O estado inicial é 1, com 0 números ímpares visitados
        return (1, 1)

    def isEnd(self, state):
        # Verifica se o estado atual é o estado final,
        # onde N é alcançado e pelo menos 3 números ímpares foram visitados
        num, odd_count = state
        if num > self.N:
            return True
        if num % 2 != 0:
            odd_count += 1

        return num == self.N and odd_count >= 3

    def succAndCost(self, state):
        num, odd_count = state
        succs = []

        if num == self.N:
            succs.append(('walk', (num + 1, odd_count), N))

        if num < self.N:
            # Verificar se o número atual é ímpar e atualizar o contador de ímpares
            if num % 2 == 0:
                odd_count += 1
            # Adicionar a ação de andar de i para i + 1
            succs.append(('walk', (num + 1, odd_count), 1))
            # Verificar se é possível usar a mágica
            if num * 2 <= self.N:
                # Verificar se o número atual é ímpar e atualizar o contador de ímpares
                if num % 2 == 0:
                    odd_count -= 1
                succs.append(('magic', (num * 2, odd_count), 2))
        return succs

In [ ]:
N = 35
p = RestrictedTransportationProblem(N)

print("DP: ",*solutionDP(p), sep='\n')

print("\nA*: ",*solutionAstar(p), sep='\n')

DP: 
('walk', (2, 1), 1)
('walk', (3, 2), 1)
('walk', (4, 2), 1)
('magic', (8, 2), 2)
('magic', (16, 2), 2)
('walk', (17, 3), 1)
('magic', (34, 3), 2)
('walk', (35, 4), 1)

A*: 
('walk', (2, 1), 1)
('magic', (4, 1), 2)
('magic', (8, 1), 2)
('magic', (16, 1), 2)
('walk', (17, 2), 1)
('magic', (34, 2), 2)
('walk', (35, 3), 1)


# Exercício 3:

As matrizes a seguir representam exemplos de mapas de rios congelados que você deve atravessar:

```
m0 =
 [
    "SFFF",
    "FHFH",
    "FFFH",
    "HFFG"
 ]

m1 =
 [
    "SFFFFFFF",
    "FFFFFFFF",
    "FFFHFFFF",
    "FFFFFHFF",
    "FFFHFFFF",
    "FHHFFFHF",
    "FHFFHFHF",
    "FFFHFFFG",
 ]
```

Nesses mapas, *S* representa a sua posição inicial; *F* significa que a posição está congelada e pode ser ocupada; *H* significa que existe um buraco nessa posição; *G* representa a posição onde você quer chegar.

Implemente a classe *FrozenLakeProblem* que recebe um mapa e implementa a mesma interface dos problemas anteriores. As ações permitidas em cada posição são: norte, sul, leste e oeste. Em sequência, encontre o algoritmo que retorna o caminho mais curto para um dado mapa.


In [ ]:
class FrozenLakeProblem:
    def __init__(self, N):
        self.N = N

    def startState(self):
        return (0, 0)

    def isEnd(self, state):
        i, j = state
        return self.N[i][j] == "G"

    def succAndCost(self, state):
        size = len(self.N[0])
        x, y = state
        costs_list = []

        if x > 0 and self.N[x - 1][y] != "H":
            new_state = (x - 1, y)
            costs_list.append(('norte', new_state, 1))
        if x < size - 1 and self.N[x + 1][y] != "H":
            new_state = (x + 1, y)
            costs_list.append(('sul', new_state, 1))
        if y > 0 and self.N[x][y - 1] != "H":
            new_state = (x, y - 1)
            costs_list.append(('oeste', new_state, 1))
        if y < size - 1 and self.N[x][y + 1] != "H":
            new_state = (x, y + 1)
            costs_list.append(('leste', new_state, 1))

        return costs_list

In [ ]:
from collections import deque

def solutionFL(problem):
    start_state = problem.startState()
    goal_state = (len(problem.N) - 1, len(problem.N[0]) - 1)

    queue = deque([(start_state, [])])
    visited = set()

    while queue:
        current_state, path = queue.popleft()

        if current_state == goal_state:
            return path

        if current_state in visited:
            continue

        visited.add(current_state)

        for action, next_state, cost in problem.succAndCost(current_state):
            new_path = path + [(action, next_state, cost)]
            queue.append((next_state, new_path))
    return None

In [ ]:
# teste:
mapa = ["SFFFHFFF",
        "FFFFHFFF",
        "FFFFHFFF",
        "HHHFHHFF",
        "FFFFFFFF",
        "FHHHHHHH",
        "FHFFHFHF",
        "FFFFFFFG",]

problem = FrozenLakeProblem(mapa)
caminho = solutionFL(problem)
print(*caminho, sep='\n')

('sul', (1, 0), 1)
('sul', (2, 0), 1)
('leste', (2, 1), 1)
('leste', (2, 2), 1)
('leste', (2, 3), 1)
('sul', (3, 3), 1)
('sul', (4, 3), 1)
('oeste', (4, 2), 1)
('oeste', (4, 1), 1)
('oeste', (4, 0), 1)
('sul', (5, 0), 1)
('sul', (6, 0), 1)
('sul', (7, 0), 1)
('leste', (7, 1), 1)
('leste', (7, 2), 1)
('leste', (7, 3), 1)
('leste', (7, 4), 1)
('leste', (7, 5), 1)
('leste', (7, 6), 1)
('leste', (7, 7), 1)


Imagine agora que, dado que o lago está congelado, quando você tenta se movimentar nele, existe uma chance de você escorregar e se movimentar em uma direção aleatória perpendicular à direção que você pretendia. Após algumas tentativas, você percebe que você escorrega com probabilidade *p* toda vez que tenta se movimentar e que, quando escorrega, é igual a chance de ir para qualquer um dos dois lados possíveis.

Implemente a classe *FrozenLakeMDP* que recebe um mapa e uma probabilidade e implementa as funções necessárias para definir a interface desse problema para utilização com os algoritmos de *policyEvaluation* e *valueIteration* apresentados em sala de aula. Em seguida, implemente os algoritmos de *policyEvaluation* e *valueIteration* nas funções com os respectivos nomes.

In [ ]:
class FrozenLakeMDP:
    def __init__(self, m, p):
        self.map = m
        self.probability = p

    def startState(self):
        return (0, 0)

    def isEnd(self, state):
        i, j = state
        return self.map[i][j] == "G"

    def states(self):
        states = []
        for i in range(len(self.map)):
            for j in range(len(self.map[i])):
                if self.map[i][j] != 'H':  # Ignora os estados que são 'H'
                    states.append((i, j))
        return states

    def actions(self, state):
        x, y = state
        actions = []
        if self.map[x][y] not in ['H', 'G']:  # Ignora os estados que são  'H' ou 'G'
            if x > 0 and self.map[x-1][y] != "H":
                actions.append(('norte', (x-1, y)))
            if x < len(self.map) - 1 and self.map[x+1][y] != "H":
                actions.append(('sul', (x+1, y)))
            if y > 0 and self.map[x][y-1] != "H":
                actions.append(('oeste', (x, y-1)))
            if y < len(self.map[0]) - 1 and self.map[x][y+1] != "H":
                actions.append(('leste', (x, y+1)))
        return actions

    def succProbReward(self, state, action):
        x, y = state
        if self.map[x][y] in ['H', 'G']:  # Se o estado é 'H' ou 'G', não há transição
            return []

        successors = []
        slip_prob = (self.probability) / 2
        remain_prob = (1 - self.probability)

        if action == 'norte' and x > 0 and self.map[x-1][y] != "H":
            new_state = (x - 1, y)
        elif action == 'sul' and x < len(self.map) - 1 and self.map[x+1][y] != "H":
            new_state = (x + 1, y)
        elif action == 'oeste' and y > 0 and self.map[x][y-1] != "H":
            new_state = (x, y - 1)
        elif action == 'leste' and y < len(self.map[0]) - 1 and self.map[x][y+1] != "H":
            new_state = (x, y + 1)
        else:
            new_state = (x, y)

        successors.append((new_state, remain_prob, -0.03))    # recomepensa negativa para achar o caminho mais rapido, pois ficar no mapa andando sem limite de tempo é ruim

        if slip_prob > 0.0:
            if action == 'norte' and y > 0 and self.map[x][y-1] != "H":
                new_state = (x, y - 1)
            elif action == 'sul' and y > 0 and self.map[x][y-1] != "H":
                new_state = (x, y - 1)
            elif action == 'oeste' and x > 0 and self.map[x-1][y] != "H":
                new_state = (x - 1, y)
            elif action == 'leste' and x > 0 and self.map[x-1][y] != "H":
                new_state = (x - 1, y)
            else:
                new_state = (x, y)

            successors.append((new_state, slip_prob, -0.03))    # recomepensa negativa para achar o caminho mais rapido, pois ficar no mapa andando sem limite de tempo é ruim

            if action == 'norte' and y < len(self.map[0]) - 1 and self.map[x][y+1] != "H":
                new_state = (x, y + 1)
            elif action == 'sul' and y < len(self.map[0]) - 1 and self.map[x][y+1] != "H":
                new_state = (x, y + 1)
            elif action == 'oeste' and x < len(self.map) - 1 and self.map[x+1][y] != "H":
                new_state = (x + 1, y)
            elif action == 'leste' and x < len(self.map) - 1 and self.map[x+1][y] != "H":
                new_state = (x + 1, y)
            else:
                new_state = (x, y)

            successors.append((new_state, slip_prob, -0.03))    # recomepensa negativa para achar o caminho mais rapido, pois ficar no mapa andando sem limite de tempo é ruim

        return successors

In [ ]:
def policyEvaluation(mdp, policy):
    gama = 1
    states = mdp.states()
    V = {s: 0 for s in states}  # Inicializa o valor de todos os estados como 0
    i = len(mdp.map) - 1
    j = len(mdp.map[0]) - 1
    V[(i,j)] = 1

    while True:
        delta = 0  # Utilizado para verificar a convergência

        for state in states:
            if mdp.isEnd(state):
                continue

            new_value = 0

            for action in mdp.actions(state):
                successors = mdp.succProbReward(state, action)
                action_value = 0

                for next_state, prob, reward in successors:
                    action_value += prob * (reward + gama * V[next_state])

                ac, stt = action
                new_value += policy[state][ac] * action_value

            delta = max(delta, abs(V[state] - new_value))
            V[state] = new_value

        if delta < 1e-6:  # Critério de convergência
            break

    return V

################################################################################
def valueIteration(mdp):
    gama = 1
    policy = []
    V = {state: 0 for state in mdp.states()}
    i = len(mdp.map) - 1
    j = len(mdp.map[0]) - 1
    V[(i,j)] = 1

    # Definição do critério de parada
    epsilon = 1e-6
    delta = float('inf')

    # Iteração até convergência
    while delta > epsilon:
        delta = 0

        V_prev = V.copy()

        # Atualização dos valores de estado
        for state in mdp.states():
            if not mdp.isEnd(state):
                max_value = float('-inf')

                # Cálculo do valor máximo entre as ações possíveis
                for action, _ in mdp.actions(state):
                    action_value = 0

                    for next_state, probability, reward in mdp.succProbReward(state, action):
                        action_value += probability * (reward + gama * V_prev[next_state])

                    # Atualização do valor máximo
                    max_value = max(max_value, action_value)

                # Verificação da diferença entre o valor atual e o novo valor
                delta = max(delta, abs(max_value - V[state]))

                # Atualização do valor de estado
                V[state] = max_value

    state = (0,0)
    while not mdp.isEnd(state):
        V_max = float('-inf')
        for action, next_state in mdp.actions(state):
            if V[next_state] > V_max:
                V_max = V[next_state]
                path_action = action
                path_state = next_state
                state = next_state

        policy.append((path_action, path_state))


    return V, policy

In [ ]:
m0 = ["SFFF",
      "FHFH",
      "FFFH",
      "HFFG"]



flp = FrozenLakeMDP(m0, 0)
V, policy = valueIteration(flp)
print("V: ", V)
print(*policy, sep='\n')

V:  {(0, 0): 0.8199999999999998, (0, 1): 0.8499999999999999, (0, 2): 0.8799999999999999, (0, 3): 0.8499999999999999, (1, 0): 0.8499999999999999, (1, 2): 0.9099999999999999, (2, 0): 0.8799999999999999, (2, 1): 0.9099999999999999, (2, 2): 0.94, (3, 1): 0.94, (3, 2): 0.97, (3, 3): 1}
('sul', (1, 0))
('sul', (2, 0))
('leste', (2, 1))
('sul', (3, 1))
('leste', (3, 2))
('leste', (3, 3))


In [ ]:
m0 = [
    "SFFF",
    "FHFH",
    "FFFH",
    "HFFG"
]

mdp = FrozenLakeMDP(m0, 0.2)

policy = {
    (0, 0): {'norte': 0.25, 'sul': 0.25, 'leste': 0.25, 'oeste': 0.25},
    (0, 1): {'norte': 0.25, 'sul': 0.25, 'leste': 0.25, 'oeste': 0.25},
    (0, 2): {'norte': 0.25, 'sul': 0.25, 'leste': 0.25, 'oeste': 0.25},
    (0, 3): {'norte': 0.25, 'sul': 0.25, 'leste': 0.25, 'oeste': 0.25},
    (1, 0): {'norte': 0.25, 'sul': 0.25, 'leste': 0.25, 'oeste': 0.25},
    (1, 1): {'norte': 0.25, 'sul': 0.25, 'leste': 0.25, 'oeste': 0.25},
    (1, 2): {'norte': 0.25, 'sul': 0.25, 'leste': 0.25, 'oeste': 0.25},
    (1, 3): {'norte': 0.25, 'sul': 0.25, 'leste': 0.25, 'oeste': 0.25},
    (2, 0): {'norte': 0.25, 'sul': 0.25, 'leste': 0.25, 'oeste': 0.25},
    (2, 1): {'norte': 0.25, 'sul': 0.25, 'leste': 0.25, 'oeste': 0.25},
    (2, 2): {'norte': 0.25, 'sul': 0.25, 'leste': 0.25, 'oeste': 0.25},
    (2, 3): {'norte': 0.25, 'sul': 0.25, 'leste': 0.25, 'oeste': 0.25},
    (3, 0): {'norte': 0.25, 'sul': 0.25, 'leste': 0.25, 'oeste': 0.25},
    (3, 1): {'norte': 0.25, 'sul': 0.25, 'leste': 0.25, 'oeste': 0.25},
    (3, 2): {'norte': 0.25, 'sul': 0.25, 'leste': 0.25, 'oeste': 0.25},
    (3, 3): {'norte': 0.25, 'sul': 0.25, 'leste': 0.25, 'oeste': 0.25}
}

values = policyEvaluation(mdp, policy)

for state, value in values.items():
    print("State:", state, "Value:", value)

State: (0, 0) Value: -0.029999999999563445
State: (0, 1) Value: -0.029999999999563445
State: (0, 2) Value: -0.0899971394602207
State: (0, 3) Value: -0.009999999999999997
State: (1, 0) Value: -0.029999999999563445
State: (1, 2) Value: -0.029999999999563445
State: (2, 0) Value: -0.029999999999563445
State: (2, 1) Value: -0.0899971394602207
State: (2, 2) Value: -0.0899971394602207
State: (3, 1) Value: -0.029999999999563445
State: (3, 2) Value: -0.0899971394602207
State: (3, 3) Value: 1


Por último, execute o algoritmo de *valueIteration* para uma mdp com *p = 0*. Observe o caminho tomado executando a política ótima obtida. Qual a relação desse caminho com o caminho mais curto obtido no item anterior?

## Resposta:

Quando executando o mdp com p = 0, a probabilidade de escorregar não existe, logo o problema se torna determinístico, o valor de cada estado será computado baseado na distância do estado ate a chegada, similar ao A*. Desta forma o caminho mínimo achado, será igual ao do exercício anterior.